<a href="https://colab.research.google.com/github/alisonnnnn88/programming_language/blob/main/HW1_%E6%8F%90%E6%AC%BE%E6%A9%9F%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ==== 歡迎使用提款機系統 ====
 📌 提款機系統使用說明

請使用下列功能選單：

> 1 輸入收入金額  
> 2 輸入支出金額  
> 3 查看目前剩餘金額  
> 4 清除所有紀錄（⚠️ 不可復原）  
> 5 離開系統

💡 小提示：支出時若金額超過餘額，會顯示「餘額不足」。

💵 提款紀錄: https://docs.google.com/spreadsheets/d/1h75idXVO6GMosL5bPUF-G76MqxOziHu1tJieRtpdkP4/edit?usp=sharing


In [ ]:
# 安裝必要套件
!pip install --upgrade gspread gspread_dataframe

# 授權登入
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from datetime import datetime
import pandas as pd

# 登入 Google Sheets API
creds, _ = default()
gc = gspread.authorize(creds)

# ✏️ 修改為你自己的試算表名稱與工作表名稱
spreadsheet_name = "我的提款機紀錄"
worksheet_name = "工作表1"

# 連接試算表
spreadsheet = gc.open(spreadsheet_name)
worksheet = spreadsheet.worksheet(worksheet_name)

# 初始化欄位
if worksheet.acell('A1').value != "日期":
    worksheet.update('A1:E1', [["日期", "時間", "收入", "支出", "剩餘金額"]])
    worksheet.resize(rows=2)

# 主程式
def atm():
    print("\n==== 歡迎使用提款機系統 ====")

    while True:
        print("\n請選擇功能：")
        print("1. 輸入收入金額")
        print("2. 輸入支出金額")
        print("3. 印出剩餘金額")
        print("4. 清除所有紀錄")
        print("5. 離開系統")

        choice = input("輸入選項 (1-5)：")

        # 讀取目前資料
        data = worksheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0]) if len(data) > 1 else pd.DataFrame(columns=data[0])
        df["收入"] = pd.to_numeric(df["收入"], errors='coerce').fillna(0)
        df["支出"] = pd.to_numeric(df["支出"], errors='coerce').fillna(0)
        df["剩餘金額"] = pd.to_numeric(df["剩餘金額"], errors='coerce').fillna(0)
        current_balance = df["剩餘金額"].iloc[-1] if not df.empty else 0

        # 收入
        if choice == "1":
            try:
                amount = float(input("請輸入收入金額："))
                if amount <= 0:
                    print("❌ 請輸入大於 0 的金額")
                    continue
            except ValueError:
                print("❌ 請輸入有效的金額")
                continue

            date_input = input("請輸入日期（格式 YYYY-MM-DD，留空則為今天）：")
            time_input = input("請輸入時間（格式 HH:MM，留空則為現在時間）：")
            now = datetime.now()
            date = date_input if date_input else now.strftime("%Y-%m-%d")
            time = time_input if time_input else now.strftime("%H:%M:%S")
            new_balance = current_balance + amount
            worksheet.append_row([date, time, amount, "", new_balance])
            print(f"✅ 已記錄收入 {amount} 元，剩餘 {new_balance} 元")

        # 支出
        elif choice == "2":
            try:
                amount = float(input("請輸入支出金額："))
                if amount <= 0:
                    print("❌ 請輸入大於 0 的金額")
                    continue
            except ValueError:
                print("❌ 請輸入有效的金額")
                continue

            if amount > current_balance:
                print("⚠️ 餘額不足，無法支出。")
                continue

            date_input = input("請輸入日期（格式 YYYY-MM-DD，留空則為今天）：")
            time_input = input("請輸入時間（格式 HH:MM，留空則為現在時間）：")
            now = datetime.now()
            date = date_input if date_input else now.strftime("%Y-%m-%d")
            time = time_input if time_input else now.strftime("%H:%M:%S")
            new_balance = current_balance - amount
            worksheet.append_row([date, time, "", amount, new_balance])
            print(f"✅ 已記錄支出 {amount} 元，剩餘 {new_balance} 元")

        # 查詢餘額
        elif choice == "3":
            print(f"💰 目前剩餘金額為：{current_balance} 元")

        # 清除所有紀錄
        elif choice == "4":
            confirm = input("⚠️ 確定要清除所有紀錄嗎？(yes/no)：")
            if confirm.lower() == "yes":
                worksheet.clear()
                worksheet.update('A1:E1', [["日期", "時間", "收入", "支出", "剩餘金額"]])
                print("✅ 所有紀錄已清除")
            else:
                print("❌ 已取消清除操作")

        # 離開
        elif choice == "5":
            print("👋 感謝使用，再見！")
            break

        else:
            print("❗ 請輸入 1～5 之間的選項")

# 啟動 ATM
atm()


==== 歡迎使用提款機系統 ====

請選擇功能：
1. 輸入收入金額
2. 輸入支出金額
3. 印出剩餘金額
4. 清除所有紀錄
5. 離開系統
輸入選項 (1-5)：2
請輸入支出金額：2000
⚠️ 餘額不足，無法支出。

請選擇功能：
1. 輸入收入金額
2. 輸入支出金額
3. 印出剩餘金額
4. 清除所有紀錄
5. 離開系統
輸入選項 (1-5)：1
請輸入收入金額：10000
請輸入日期（格式 YYYY-MM-DD，留空則為今天）：2025-09-11
請輸入時間（格式 HH:MM，留空則為現在時間）：
✅ 已記錄收入 10000.0 元，剩餘 10000.0 元

請選擇功能：
1. 輸入收入金額
2. 輸入支出金額
3. 印出剩餘金額
4. 清除所有紀錄
5. 離開系統
輸入選項 (1-5)：2
請輸入支出金額：300
請輸入日期（格式 YYYY-MM-DD，留空則為今天）：
請輸入時間（格式 HH:MM，留空則為現在時間）：
✅ 已記錄支出 300.0 元，剩餘 9700.0 元

請選擇功能：
1. 輸入收入金額
2. 輸入支出金額
3. 印出剩餘金額
4. 清除所有紀錄
5. 離開系統
輸入選項 (1-5)：3
💰 目前剩餘金額為：9700 元

請選擇功能：
1. 輸入收入金額
2. 輸入支出金額
3. 印出剩餘金額
4. 清除所有紀錄
5. 離開系統
輸入選項 (1-5)：5
👋 感謝使用，再見！
